In [ ]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

In [ ]:
from tqdm import tqdm
import cv2 as cv
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

hand_signs = os.listdir("data/train")

images = []
labels = []

for sign in hand_signs:
    path = "data/train/" + sign
    
    for image in tqdm(os.listdir(path), desc=f"Loading images for {sign}"):
        img = cv.imread(path + "/" + image)
        img = cv.resize(img, (380, 380))
        if img is not None:
            images.append(img)
            labels.append(sign)
    
print("\nConverting training images to np array with float32 values scaled between 0.0 to 1.0")
X = np.array(images).astype("float32") / 255.0

print("\nEncoding Y properly to a understandable format")
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

print("\nOne Hot Encoding training labels")
y = y.reshape(-1, 1)
one_hot_encoder = OneHotEncoder()
Y = one_hot_encoder.fit_transform(y)

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout

def create_vgg_model(input_shape=(380,380,3), num_classes=13):
    with tf.device("/gpu:0"):
        vgg_model = VGG16(
            include_top=False,
            input_shape=input_shape,
            pooling="avg",
            weights="imagenet"
        )
        
        for layer in vgg_model.layers[:-4]:
            layer.trainable = False
        
        x = Flatten()(vgg_model.output)
        x = Dropout(0.3)(x)
        x = Dense(4096, activation="relu", name="fc1")(x)
        x = Dropout(0.3)(x)
        x = Dense(1024, activation="relu", name="fc2")(x)
        x = Dropout(0.3)(x)
        output = Dense(num_classes, activation="softmax", name="predictions")(x)
        
        model = Model(inputs=vgg_model.input, outputs=output, name='Sharingan')
        
        return model

In [ ]:
hand_signs = os.listdir("data/test")

images_test = []
labels_test = []

for sign in hand_signs:
    path = "data/test/" + sign
    
    for image in tqdm(os.listdir(path), desc=f"Loading images for {sign}"):
        img = cv.imread(path + "/" + image)
        img = cv.resize(img, (380, 380))
        if img is not None:
            images_test.append(img)
            labels_test.append(sign)
    
print("\nConverting training images to np array with float32 values scaled between 0.0 to 1.0")
X_test = np.array(images_test).astype("float32") / 255.0

print("\nEncoding Y properly to a understandable format")
label_encoder = LabelEncoder()
y_test = label_encoder.fit_transform(labels_test)

print("\nOne Hot Encoding training labels")
y_test = y_test.reshape(-1, 1)
one_hot_encoder = OneHotEncoder()
Y_test = one_hot_encoder.fit_transform(y_test)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scipy.sparse import issparse

def train_model(model, X, Y, X_test, Y_test):
    if issparse(X):
        X = X.toarray()
    if issparse(Y):
        Y = Y.toarray() 
    if issparse(X_test):
        X_test = X_test.toarray() 
    if issparse(Y_test):
        Y_test = Y_test.toarray() 
    
    X = np.asarray(X)
    Y = np.asarray(Y)
    X_test = np.asarray(X_test)
    Y_test = np.asarray(Y_test)
    
    train_datagen = ImageDataGenerator(
        featurewise_center=False,
        featurewise_std_normalization=False,
        rotation_range=25,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=False,
    )
    
    val_datagen = ImageDataGenerator()

    model.compile(
        loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        metrics=['accuracy']
    )
    
    batch_size = 8
    
    callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.5, 
            patience=3, 
            min_lr=1e-5
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=5, 
            restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            './models/best_VGG_Classifier.keras', 
            save_best_only=True, 
            monitor='val_accuracy'
        )
    ]
    
    train_gen = train_datagen.flow(X, Y, batch_size=batch_size)
    val_gen = val_datagen.flow(X_test, Y_test, batch_size=batch_size)
    
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        steps_per_epoch=len(X) // batch_size,
        validation_steps=len(X_test) // batch_size,
        epochs=50,
        callbacks=callbacks,
    )
    
    return history


In [ ]:
model = create_vgg_model(num_classes=len(hand_signs))

model.summary()

history = train_model(model, X, Y, X_test, Y_test)

In [ ]:
model.save("./models/final_VGG_Classifier.keras")